In [104]:
import requests
import json
import pandas as pd
import time
import random

In [105]:
from datetime import datetime, timedelta

# Current time in UTC
now = datetime.utcnow()

# Calculate the start time and end time
start_time = now.replace(minute=0, second=0, microsecond=0)
start_time = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
end_time = (now + timedelta(weeks=1)).strftime("%Y-%m-%dT%H:%M:%SZ")  # 2 weeks later
end_time = (now + timedelta(weeks=2)).replace(
        hour=23,
        minute=59,
        second=59,
        microsecond=0
    )
end_time = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")

# Construct the URL
print(start_time)
print(end_time)

2025-02-03T12:00:00Z
2025-02-17T23:59:59Z


In [91]:
type(end_time)

str

In [92]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store each batch of matches data
all_matches = []

# Loop over drillDownTagIds ranges in steps of 900
for start in range(200, 14000, 200):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        f"startTimeFrom={start_time}"
        f"startTimeTo={end_time}&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        for event in data.get('data', {}).get('events', []):
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)
            
            matches.append(match)

        # Add the batch of matches to the all_matches list
        all_matches.extend(matches)

        # # Sleep for a random duration between 1 and 4 seconds, with variability of 0.1 ms
        # time.sleep(random.uniform(0, 1.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 200}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the resulting DataFrame
print(final_matches_df)

An error occurred for drilldown ID range 12800–13000: 'NoneType' object is not iterable
     event_id                                         match_name  \
0     5951240                      Real Betis vs Athletic Bilbao   
1     6491226                              CF Estrela vs Benfica   
2     6491229                    Sporting Cp vs Sporting Farense   
3     6611772                         Chelsea vs West Ham United   
4     6296228                     Bayer Leverkusen vs 1. FC Koln   
...       ...                                                ...   
2223  6737885  South Australian Scorpions (W) vs Queensland F...   
2224  6736590                Tasmanian Tigers (W) vs Act Meteors   
2225  6650138                  Tom Nolan vs Viacheslav Borshchev   
2226  6239787                      Weili Zhang vs Tatiana Suarez   
2227  6643563                  Zsc Lions Zurich vs Farjestads BK   

                start_time                       home_team  \
0     2025-02-02T20:00:00Z       

In [4]:
len(final_matches_df)

2218

In [5]:
final_matches_df['event_id'].nunique()

1418

In [6]:
final_matches_df[final_matches_df['match_name'].str.contains('PSV')]

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
505,6351955,NEC vs PSV,2025-02-01T20:00:00Z,NEC,PSV,Nederland Eredivisie,Nederland,Voetbal,7.00,1.36
513,6626894,Jong PSV vs VVV Venlo,2025-01-27T19:00:00Z,Jong PSV,VVV Venlo,Nederland Eerste Divisie,Nederland,Voetbal,2.40,2.65
520,6682208,PSV Eindhoven vs FC Twente,2025-02-01T15:30:00Z,PSV Eindhoven,FC Twente,Nederland Eredivisie Vrouwen,Nederland,Voetbal,1.36,5.75
1153,6248696,PSV vs Liverpool,2025-01-29T20:00:00Z,PSV,Liverpool,UEFA Champions League,Europees,Voetbal,2.80,2.37


In [7]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []

# Set batch size
batch_size = 30

# Loop over event_ids in batches of 500
for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        # Get a batch of event_ids
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        
        # Concatenate event_ids for the URL
        event_ids_str = ','.join(map(str, batch_ids))
        
        # Define the URL for the current batch
        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&"
            "includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"
        )
        
        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }
        
        # Send request
        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Initialize lists to store the data for this batch
        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        # Process the JSON response for each event
        for event in json_data['data']['events']:  # Iterate over each event
            event_id = event['id']
            event_name = event['name']
            
            for market in event['markets']:  # Iterate over each market in the event
                market_name = market['name']
                
                for outcome in market['outcomes']:  # Iterate over outcomes for each market
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']
                    
                    # Append captured values to the respective lists
                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        # Create a temporary DataFrame for this batch and append it to all_data
        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })
        
        all_data.append(temp_df)
        
        # Wait between .. and .. seconds
        time.sleep(random.uniform(0.02, 1.81))
    except:
        None

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

       event_id                                       Event Name  \
0       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
1       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
2       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
3       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
4       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
...         ...                                              ...   
115813  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115814  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115815  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115816  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115817  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   

                Market Name                 Outcome Name  Odds (Decimal)  \
0               Draw No Bet               TSG Hoffenheim            2.41   
1               Draw No Bet    

In [8]:
final_df[final_df['Event Name'].str.contains('PSV')]['Event Name'].unique()

array(['NEC vs PSV', 'Jong PSV vs VVV Venlo',
       'PSV Eindhoven vs FC Twente', 'PSV vs Liverpool'], dtype=object)

In [9]:
# Perform the left join on 'event_id' to get 'sport' and 'group_name' from events_df
final_df = final_df.merge(final_matches_df[['event_id', 'sport', 'competition']], on='event_id', how='left').drop_duplicates()
# Remove columns that end with '_x'
final_df = final_df[[col for col in final_df.columns if not col.endswith('_x')]]

# Rename columns to remove '_y'
final_df.columns = final_df.columns.str.replace('_y', '', regex=False)

In [10]:
# Replace 'A' with 2 and 'H' with 1 in the outcome_subtype column
final_df['Outcome SubType'] = final_df['Outcome SubType'].replace({'A': '2', 'H': '1'})

# Display the updated final_df
print(final_df)

       event_id                                       Event Name  \
0       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
1       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
2       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
3       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
4       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
...         ...                                              ...   
175067  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175068  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175069  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175070  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175071  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   

                Market Name                 Outcome Name  Odds (Decimal)  \
0               Draw No Bet               TSG Hoffenheim            2.41   
1               Draw No Bet    

In [11]:
set(final_df[final_df['sport'].str.contains('Voetbal')].competition)

{'Andorra First Division',
 'Argentinië Superliga',
 'Australië A-League',
 'Australië W-League',
 'Azerbeidzjan Premier League',
 'België 1B Pro League',
 'België Jupiler Pro League',
 'Brazil Campeonato Paranaense',
 'Brazil Gaucho A1',
 'Brazil Supercup',
 'Brazilië Cearense',
 'Brazilië Mineiro Modulo I',
 'Brazilië Paulista A1\n',
 'Colombia Categoría Primera A',
 'Costa Rica Ascenso League',
 'Costa Rica Primera Division',
 'Cyprus Beker',
 'Cyprus First Division',
 'Duitsland 2.Bundesliga',
 'Duitsland 3.Liga',
 'Duitsland Bundesliga',
 'Egypte Premier League',
 'Egypte Second Division',
 'El Salvador Primera Division',
 'Engeland Championship',
 'Engeland FA Super League Vrouwen',
 'Engeland League 1',
 'Engeland League 2',
 'Engeland National League',
 'Engeland National League Noord',
 'Engeland National League Zuid',
 'Engeland Premier League',
 'Ethiopië Premier League',
 'Europa Super League',
 'Europees Kampioenschap',
 'Frankrijk Ligue 1',
 'Frankrijk Ligue 2',
 'Frankri

In [12]:
final_df[(final_df['Event Name'].str.contains('Milan')) & final_df['sport']=='Voetbal']

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition


In [13]:
[i for i in list(set(final_df['Market Name'])) if 'No Bet' in i]

['1e Helft: Draw No Bet',
 '3e Periode - Draw No Bet',
 '2e Helft: Draw No Bet',
 '2e Periode - Draw No Bet',
 'Draw No Bet',
 '1e Periode - Draw No Bet']

In [14]:
final_output_name = 'totoAllSports' + start_time + '.csv'
final_df.to_csv(final_output_name)

## Check id ranges

In [106]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store all match data
all_matches = []

# Initialize a dictionary to store sports and competitions by drillDownTagIds range
range_summary = {}

# Loop over drillDownTagIds ranges in steps of 1000
for start in range(0, 14000, 20):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        "startTimeFrom=2025-01-30T21:00:00Z&"
        "startTimeTo=2025-02-30T22:59:59Z&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        sports_set = set()
        competitions_set = set()

        for event in data.get('data', {}).get('events', []):
            # Collect match data
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Collect sports and competitions
            sports_set.add(match["sport"])
            competitions_set.add(match["competition"])

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)

            matches.append(match)

        # Add matches to the all_matches list
        all_matches.extend(matches)

        # Add sports and competitions to the range summary
        range_summary[f"{start}-{start + 20}"] = {
            "sports": list(sports_set),
            "competitions": list(competitions_set)
        }

        # Sleep for a random duration between 1 and 2 seconds
        # time.sleep(random.uniform(1, 2.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 20}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the range summary
for range_key, summary in range_summary.items():
    print(f"\nDrillDownTagIds Range: {range_key}")
    print(f"Sports: {summary['sports']}")
    print(f"Competitions: {summary['competitions']}")

# Save the match data to a CSV or Excel file if needed
final_matches_df.to_csv("matches_output.csv", index=False)

# Optionally, save the range summary to a file
pd.DataFrame.from_dict(range_summary, orient="index").to_csv(f"range_summary_{datetime.utcnow()}.csv")

An error occurred for drilldown ID range 0–20: 'NoneType' object is not iterable
An error occurred for drilldown ID range 500–520: 'NoneType' object is not iterable
An error occurred for drilldown ID range 980–1000: 'NoneType' object is not iterable

DrillDownTagIds Range: 20-40
Sports: ['Tennis', 'Voetbal', 'Football', 'IJshockey']
Competitions: ['WTA Abu Dhabi, Enkelspel', 'WTA 125K Mumbai, Enkelspel', 'Duitsland DFB Pokal', 'Engeland EFL Trophy', 'VS NHL', 'WTA Abu Dhabi, Doubles', 'WTA Cluj Napoca, Doubles', 'Duitsland Bundesliga', 'Engeland Championship', 'Frankrijk Beker', 'Spanje LaLiga', 'Engeland FA Cup', 'Spanje Copa del Rey', 'Frankrijk Ligue 1', 'England League Cup', 'Engeland Premier League', 'Italië Coppa Italia', 'WTA 125K Mumbai, Doubles', 'WTA Cluj Napoca, Enkelspel', 'Spanje Segunda Division', 'NFL', 'Italië Serie A']

DrillDownTagIds Range: 40-60
Sports: ['Tennis', 'Voetbal', 'Football', 'IJshockey']
Competitions: ['WTA Abu Dhabi, Enkelspel', 'WTA 125K Mumbai, Enkels

In [6]:
# Optionally, save the range summary to a file
pd.DataFrame.from_dict(range_summary, orient="index").to_csv(f"range_summary_{datetime.utcnow()}.csv")

In [19]:
# keys_with_basketbal = [key for key, value in range_summary.items() if 'Voetbal' in value.get('sports', [])]
# keys_with_basketbal

In [64]:
final_matches_df = final_matches_df[final_matches_df['match_name'].str.contains('AS Saint-Etienne vs Stade Rennes')]

In [65]:
final_matches_df

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
2674,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
2864,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3054,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3244,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3435,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3625,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3815,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
4030,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
4220,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
4410,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17


In [86]:
import requests
import time
import random
import pandas as pd

all_data = []
batch_size = 30

for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        event_ids_str = ','.join(map(str, batch_ids))

        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=true&includeCommentary=true&includeIncidents=true&includeRace=true&"
            "includePools=true&includeNonFixedOdds=true&lang=nl-NL&channel=I"
        )

        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }

        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Lijst om resultaten per event op te slaan
        event_market_data = []

        for event in json_data['data']['events']:
            event_id = event['id']
            event_name = event['name']

            # Filter de juiste marketIds uit de collectie met naam 'Alles'
            market_ids = []
            for collection in event.get('collections', []):
                if collection['name'] == 'Alles':
                    market_ids = collection.get('marketIds', [])
                    break  # Stop zodra we de juiste collectie vinden

            # Voeg alleen event_id en marketIds toe
            event_market_data.append({
                'event_id': event_id,
                'Event Name': event_name,
                'MarketIDs': market_ids
            })

        if event_market_data:
            temp_df = pd.DataFrame(event_market_data)
            all_data.append(temp_df)

    except Exception as e:
        print(f"Error: {e}")

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df)
else:
    print("No data found")

  event_id                        Event Name  \
0  6612005  AS Saint-Etienne vs Stade Rennes   

                                           MarketIDs  
0  [159429586, 159429587, 159429588, 159429737, 1...  


In [72]:
len(temp_df['MarketIDs'].iloc[0])

19

In [84]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []

# Set batch size
batch_size = 30

# Loop over event_ids in batches of 500
for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        # Get a batch of event_ids
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        
        # Concatenate event_ids for the URL
        event_ids_str = ','.join(map(str, batch_ids))
        
        url = (
            "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            "marketIds=159817403,159817408,159817409,159817435,159817436,159817437,159817438,159817440,159817441,159817442,159817443,159817444,159817451,159817456,159817462,159817466,159817471,159817474,159817475,159817476,159817477,159817481,159817482,159817483,159817484,159817485,159817486,159817487,159817488,159817491,159817493,159817494,159817497,159817499,159817503,159817507,159817509,159817517,159817520,159817523,159817524,159817527,159817544,159817549,159817550,159817551,159860158,159860161,159860162,159860164,159860166,159860167,159860168,159860169,159860170,159860172,159860173,159860174,159860175,159860176,159860177,159860179,159860180,159860181,159860183,159860184,159860185,159860186,159860188,159860189&"
            "includeChildMarkets=true&"
            "includePriceHistory=false&"
            "includeCommentary=false&"
            "includeMedia=false&"
            "includePoolsInfo=false&"
            "includeNonFixedOdds=false&"
            "lang=nl-NL&"
            "channel=I"
            )
        
        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }
        
        # Send request
        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Initialize lists to store the data for this batch
        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        # Process the JSON response for each event
        for event in json_data['data']['events']:  # Iterate over each event
            event_id = event['id']
            event_name = event['name']
            
            for market in event['markets']:  # Iterate over each market in the event
                market_name = market['name']
                
                for outcome in market['outcomes']:  # Iterate over outcomes for each market
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']
                    
                    # Append captured values to the respective lists
                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        # Create a temporary DataFrame for this batch and append it to all_data
        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })
        
        all_data.append(temp_df)
        
        # Wait between .. and .. seconds
        time.sleep(random.uniform(0.02, 1.81))
    except:
        None

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

    event_id                Event Name  \
0    6608244  Strasbourg vs Angers SCO   
1    6608244  Strasbourg vs Angers SCO   
2    6608244  Strasbourg vs Angers SCO   
3    6608244  Strasbourg vs Angers SCO   
4    6608244  Strasbourg vs Angers SCO   
..       ...                       ...   
276  6608244  Strasbourg vs Angers SCO   
277  6608244  Strasbourg vs Angers SCO   
278  6608244  Strasbourg vs Angers SCO   
279  6608244  Strasbourg vs Angers SCO   
280  6608244  Strasbourg vs Angers SCO   

                                     Market Name  \
0                Resultaat en Beide Teams Scoren   
1                Resultaat en Beide Teams Scoren   
2                Resultaat en Beide Teams Scoren   
3                Resultaat en Beide Teams Scoren   
4                Resultaat en Beide Teams Scoren   
..                                           ...   
276  Dubbele Kans en Aantal Goals Over/Under 4.5   
277  Dubbele Kans en Aantal Goals Over/Under 4.5   
278  Dubbele Kans en Aantal

In [85]:
final_df

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType
0,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Angers SCO en Nee,7.50,13,2,--,None
1,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Angers SCO en Ja,8.50,15,2,--,None
2,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Gelijkspel en Nee,10.00,9,1,--,None
3,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Strasbourg en Ja,3.65,53,20,--,None
4,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Gelijkspel en Ja,4.50,7,2,--,None
...,...,...,...,...,...,...,...,...,...
276,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Gelijkspel of Angers SCO en Over,19.00,18,1,--,None
277,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Strasbourg of Gelijkspel en Over,7.00,6,1,--,None
278,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Gelijkspel of Angers SCO en Under,1.91,91,100,--,None
279,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Strasbourg of Angers SCO en Over,6.25,21,4,--,None


In [82]:
url = (
    "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
    "marketIds=159817403,159817408,159817409,159817435,159817436,159817437,159817438,159817440,159817441,159817442,159817443,159817444,159817451,159817456,159817462,159817466,159817471,159817474,159817475,159817476,159817477,159817481,159817482,159817483,159817484,159817485,159817486,159817487,159817488,159817491,159817493,159817494,159817497,159817499,159817503,159817507,159817509,159817517,159817520,159817523,159817524,159817527,159817544,159817549,159817550,159817551,159860158,159860161,159860162,159860164,159860166,159860167,159860168,159860169,159860170,159860172,159860173,159860174,159860175,159860176,159860177,159860179,159860180,159860181,159860183,159860184,159860185,159860186,159860188,159860189&"
    "includeChildMarkets=true&"
    "includePriceHistory=false&"
    "includeCommentary=false&"
    "includeMedia=false&"
    "includePoolsInfo=false&"
    "includeNonFixedOdds=false&"
    "lang=nl-NL&"
    "channel=I"
    )

headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
}

# Send request
response = requests.get(url, headers=headers)
json_data = response.json()

In [100]:
final_matches_df = final_matches_df.drop_duplicates()

In [103]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []
batch_size = 1

# Stap 1: Haal marketIds op voor elk event
event_market_ids_dict = {}

for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        event_ids_str = ','.join(map(str, batch_ids))

        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=true&includeCommentary=true&includeIncidents=true&includeRace=true&"
            "includePools=true&includeNonFixedOdds=true&lang=nl-NL&channel=I"
        )

        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }

        response = requests.get(url, headers=headers)
        json_data = response.json()

        for event in json_data['data']['events']:
            event_id = event['id']

            # Haal alleen de marketIds op van collectie 'Alles'
            market_ids = []
            for collection in event.get('collections', []):
                if collection['name'] == 'Alles':
                    market_ids = collection.get('marketIds', [])
                    break

            # Opslaan in dictionary
            event_market_ids_dict[event_id] = market_ids

        # time.sleep(random.uniform(0.02, 1.81))

    except Exception as e:
        print(f"Error bij ophalen van marketIds: {e}. Bij event_ids: {batch_ids}")

# Stap 2: Gebruik de opgehaalde marketIds in de tweede request
for event_id, market_ids in event_market_ids_dict.items():
    if not market_ids:
        continue  # Sla over als er geen marketIds zijn

    market_ids_str = ','.join(map(str, market_ids))

    url = (
        f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
        f"marketIds={market_ids_str}&"
        "includeChildMarkets=true&"
        "includePriceHistory=false&"
        "includeCommentary=false&"
        "includeMedia=false&"
        "includePoolsInfo=false&"
        "includeNonFixedOdds=false&"
        "lang=nl-NL&"
        "channel=I"
    )

    try:
        response = requests.get(url, headers=headers)
        json_data = response.json()

        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        for event in json_data['data']['events']:
            event_name = event['name']

            for market in event['markets']:
                market_name = market['name']

                for outcome in market['outcomes']:
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']

                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })

        all_data.append(temp_df)

        # time.sleep(random.uniform(0.02, 1.81))

    except Exception as e:
        print(f"Error bij ophalen van odds data: {e}")

# Stap 3: Combineer alle opgehaalde data
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df)
else:
    print("No data found")

Error bij ophalen van odds data: list index out of range
       event_id                         Event Name  \
0       5951240      Real Betis vs Athletic Bilbao   
1       5951240      Real Betis vs Athletic Bilbao   
2       5951240      Real Betis vs Athletic Bilbao   
3       5951240      Real Betis vs Athletic Bilbao   
4       5951240      Real Betis vs Athletic Bilbao   
...         ...                                ...   
222051  6643563  Zsc Lions Zurich vs Farjestads BK   
222052  6643563  Zsc Lions Zurich vs Farjestads BK   
222053  6643563  Zsc Lions Zurich vs Farjestads BK   
222054  6643563  Zsc Lions Zurich vs Farjestads BK   
222055  6643563  Zsc Lions Zurich vs Farjestads BK   

                                              Market Name  \
0                         Resultaat en Scoort Eerste Goal   
1                         Resultaat en Scoort Eerste Goal   
2                         Resultaat en Scoort Eerste Goal   
3                         Resultaat en Scoort Eers

In [ ]:
final_df

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType
0,6612005,AS Saint-Etienne vs Stade Rennes,Dubbele Kans,Gelijkspel of Stade Rennes,1.36,9,25,DC,2
1,6612005,AS Saint-Etienne vs Stade Rennes,Dubbele Kans,AS Saint-Etienne of Gelijkspel,1.64,16,25,DC,1
2,6612005,AS Saint-Etienne vs Stade Rennes,Dubbele Kans,AS Saint-Etienne of Stade Rennes,1.33,33,100,DC,3
3,6612005,AS Saint-Etienne vs Stade Rennes,Resultaat,AS Saint-Etienne,3.20,11,5,MR,H
4,6612005,AS Saint-Etienne vs Stade Rennes,Resultaat,Gelijkspel,3.45,49,20,MR,D
...,...,...,...,...,...,...,...,...,...
380,6612005,AS Saint-Etienne vs Stade Rennes,Half-Time / Full-Time Correct Score,Gelijkspel 0-0 - AS Saint-Etienne 2-0,40.00,39,1,--,None
381,6612005,AS Saint-Etienne vs Stade Rennes,Half-Time / Full-Time Correct Score,AS Saint-Etienne 1-0 - AS Saint-Etienne 3-0,80.00,79,1,--,None
382,6612005,AS Saint-Etienne vs Stade Rennes,Half-Time / Full-Time Correct Score,Gelijkspel 0-0 - AS Saint-Etienne 1-0,13.50,100,8,--,None
383,6612005,AS Saint-Etienne vs Stade Rennes,Half-Time / Full-Time Correct Score,AS Saint-Etienne 1-0 - Stade Rennes 2-1,45.00,44,1,--,None
